# Import

In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.insert(1, parent_dir)

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
import seaborn as sns
import warnings
from tools.functions import missing_values_summary, pct_null_buckets

# Data

In [2]:
csv_path = os.path.join(parent_dir, "data")
file = 'application_test.csv'
path = os.path.join('..','data', file)
data = pd.read_csv(path)
data.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 121 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 45.0+ MB


In [7]:
data['CODE_GENDER'].dtype

dtype('O')

In [8]:
data['NAME_CONTRACT_TYPE'].dtype

dtype('O')

In [15]:
(data['AMT_INCOME_TOTAL'].dtype == 'float64') & (data['AMT_INCOME_TOTAL'].dtype == 'float64')

True

## Kernel preprocessing

In [35]:
# Kernel preprocessing for application_test
warnings.simplefilter(action='ignore', category=FutureWarning)

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows = None, nan_as_category = False):
    # Read data and merge
    df = pd.read_csv('../data/application_test.csv', nrows= num_rows)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    # del test_df
    gc.collect()

    # features = ['SK_ID_CURR', 'DAYS_EMPLOYED_PERC', 'INCOME_CREDIT_PERC', 'INCOME_PER_PERSON', 'ANNUITY_INCOME_PERC', 'PAYMENT_RATE']
    # cols = cat_cols + features

    return df

# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = pd.read_csv('../data/bureau.csv', nrows = num_rows)
    bb = pd.read_csv('../data/bureau_balance.csv', nrows = num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

# Preprocess previous_applications.csv
def previous_applications(num_rows = None, nan_as_category = True):
    prev = pd.read_csv('../data/previous_application.csv', nrows = num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows = None, nan_as_category = True):
    pos = pd.read_csv('../data/POS_CASH_balance.csv', nrows = num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg
    
# Preprocess installments_payments.csv
def installments_payments(num_rows = None, nan_as_category = True):
    ins = pd.read_csv('../data/installments_payments.csv', nrows = num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    return ins_agg

# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = pd.read_csv('../data/credit_card_balance.csv', nrows = num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    return cc_agg

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 200, early_stopping_rounds= 200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    display_importances(feature_importance_df)
    return feature_importance_df

# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')


def main(debug = False):
    num_rows = 10000 if debug else None
    df = application_train_test(num_rows)
    with timer("Process bureau and bureau_balance"):
        bureau = bureau_and_balance(num_rows)
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how='left', on='SK_ID_CURR')
        del bureau
        gc.collect()
    with timer("Process previous_applications"):
        prev = previous_applications(num_rows)
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()
    with timer("Process POS-CASH balance"):
        pos = pos_cash(num_rows)
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()
    with timer("Process installments payments"):
        ins = installments_payments(num_rows)
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()
    with timer("Process credit card balance"):
        cc = credit_card_balance(num_rows)
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()
    
    return df

In [53]:
data = main()

Bureau df shape: (305811, 116)
Process bureau and bureau_balance - done in 20s
Previous applications df shape: (338857, 249)
Process previous_applications - done in 26s
Pos-cash balance df shape: (337252, 18)
Process POS-CASH balance - done in 14s
Installments payments df shape: (339587, 26)
Process installments payments - done in 36s
Credit card balance df shape: (103558, 141)
Process credit card balance - done in 29s


In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48744 entries, 0 to 48743
Columns: 794 entries, SK_ID_CURR to CC_COUNT
dtypes: float64(621), int64(42), uint8(131)
memory usage: 253.0 MB


## Columns

In [55]:
cols_to_keep = ['SK_ID_CURR',
#  'TARGET',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'DAYS_LAST_PHONE_CHANGE',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'NAME_CONTRACT_TYPE_Cash loans',
 'NAME_CONTRACT_TYPE_Revolving loans',
 'NAME_TYPE_SUITE_Children',
 'NAME_TYPE_SUITE_Family',
 'NAME_TYPE_SUITE_Group of people',
 'NAME_TYPE_SUITE_Other_A',
 'NAME_TYPE_SUITE_Other_B',
 'NAME_TYPE_SUITE_Spouse, partner',
 'NAME_TYPE_SUITE_Unaccompanied',
 'NAME_INCOME_TYPE_Businessman',
 'NAME_INCOME_TYPE_Commercial associate',
#  'NAME_INCOME_TYPE_Maternity leave',
 'NAME_INCOME_TYPE_Pensioner',
 'NAME_INCOME_TYPE_State servant',
 'NAME_INCOME_TYPE_Student',
 'NAME_INCOME_TYPE_Unemployed',
 'NAME_INCOME_TYPE_Working',
 'NAME_EDUCATION_TYPE_Academic degree',
 'NAME_EDUCATION_TYPE_Higher education',
 'NAME_EDUCATION_TYPE_Incomplete higher',
 'NAME_EDUCATION_TYPE_Lower secondary',
 'NAME_EDUCATION_TYPE_Secondary / secondary special',
 'NAME_FAMILY_STATUS_Civil marriage',
 'NAME_FAMILY_STATUS_Married',
 'NAME_FAMILY_STATUS_Separated',
 'NAME_FAMILY_STATUS_Single / not married',
#  'NAME_FAMILY_STATUS_Unknown',
 'NAME_FAMILY_STATUS_Widow',
 'NAME_HOUSING_TYPE_Co-op apartment',
 'NAME_HOUSING_TYPE_House / apartment',
 'NAME_HOUSING_TYPE_Municipal apartment',
 'NAME_HOUSING_TYPE_Office apartment',
 'NAME_HOUSING_TYPE_Rented apartment',
 'NAME_HOUSING_TYPE_With parents',
 'OCCUPATION_TYPE_Accountants',
 'OCCUPATION_TYPE_Cleaning staff',
 'OCCUPATION_TYPE_Cooking staff',
 'OCCUPATION_TYPE_Core staff',
 'OCCUPATION_TYPE_Drivers',
 'OCCUPATION_TYPE_HR staff',
 'OCCUPATION_TYPE_High skill tech staff',
 'OCCUPATION_TYPE_IT staff',
 'OCCUPATION_TYPE_Laborers',
 'OCCUPATION_TYPE_Low-skill Laborers',
 'OCCUPATION_TYPE_Managers',
 'OCCUPATION_TYPE_Medicine staff',
 'OCCUPATION_TYPE_Private service staff',
 'OCCUPATION_TYPE_Realty agents',
 'OCCUPATION_TYPE_Sales staff',
 'OCCUPATION_TYPE_Secretaries',
 'OCCUPATION_TYPE_Security staff',
 'OCCUPATION_TYPE_Waiters/barmen staff',
 'WEEKDAY_APPR_PROCESS_START_FRIDAY',
 'WEEKDAY_APPR_PROCESS_START_MONDAY',
 'WEEKDAY_APPR_PROCESS_START_SATURDAY',
 'WEEKDAY_APPR_PROCESS_START_SUNDAY',
 'WEEKDAY_APPR_PROCESS_START_THURSDAY',
 'WEEKDAY_APPR_PROCESS_START_TUESDAY',
 'WEEKDAY_APPR_PROCESS_START_WEDNESDAY',
 'ORGANIZATION_TYPE_Advertising',
 'ORGANIZATION_TYPE_Agriculture',
 'ORGANIZATION_TYPE_Bank',
 'ORGANIZATION_TYPE_Business Entity Type 1',
 'ORGANIZATION_TYPE_Business Entity Type 2',
 'ORGANIZATION_TYPE_Business Entity Type 3',
 'ORGANIZATION_TYPE_Cleaning',
 'ORGANIZATION_TYPE_Construction',
 'ORGANIZATION_TYPE_Culture',
 'ORGANIZATION_TYPE_Electricity',
 'ORGANIZATION_TYPE_Emergency',
 'ORGANIZATION_TYPE_Government',
 'ORGANIZATION_TYPE_Hotel',
 'ORGANIZATION_TYPE_Housing',
 'ORGANIZATION_TYPE_Industry: type 1',
 'ORGANIZATION_TYPE_Industry: type 10',
 'ORGANIZATION_TYPE_Industry: type 11',
 'ORGANIZATION_TYPE_Industry: type 12',
 'ORGANIZATION_TYPE_Industry: type 13',
 'ORGANIZATION_TYPE_Industry: type 2',
 'ORGANIZATION_TYPE_Industry: type 3',
 'ORGANIZATION_TYPE_Industry: type 4',
 'ORGANIZATION_TYPE_Industry: type 5',
 'ORGANIZATION_TYPE_Industry: type 6',
 'ORGANIZATION_TYPE_Industry: type 7',
 'ORGANIZATION_TYPE_Industry: type 8',
 'ORGANIZATION_TYPE_Industry: type 9',
 'ORGANIZATION_TYPE_Insurance',
 'ORGANIZATION_TYPE_Kindergarten',
 'ORGANIZATION_TYPE_Legal Services',
 'ORGANIZATION_TYPE_Medicine',
 'ORGANIZATION_TYPE_Military',
 'ORGANIZATION_TYPE_Mobile',
 'ORGANIZATION_TYPE_Other',
 'ORGANIZATION_TYPE_Police',
 'ORGANIZATION_TYPE_Postal',
 'ORGANIZATION_TYPE_Realtor',
 'ORGANIZATION_TYPE_Religion',
 'ORGANIZATION_TYPE_Restaurant',
 'ORGANIZATION_TYPE_School',
 'ORGANIZATION_TYPE_Security',
 'ORGANIZATION_TYPE_Security Ministries',
 'ORGANIZATION_TYPE_Self-employed',
 'ORGANIZATION_TYPE_Services',
 'ORGANIZATION_TYPE_Telecom',
 'ORGANIZATION_TYPE_Trade: type 1',
 'ORGANIZATION_TYPE_Trade: type 2',
 'ORGANIZATION_TYPE_Trade: type 3',
 'ORGANIZATION_TYPE_Trade: type 4',
 'ORGANIZATION_TYPE_Trade: type 5',
 'ORGANIZATION_TYPE_Trade: type 6',
 'ORGANIZATION_TYPE_Trade: type 7',
 'ORGANIZATION_TYPE_Transport: type 1',
 'ORGANIZATION_TYPE_Transport: type 2',
 'ORGANIZATION_TYPE_Transport: type 3',
 'ORGANIZATION_TYPE_Transport: type 4',
 'ORGANIZATION_TYPE_University',
 'ORGANIZATION_TYPE_XNA',
 'FONDKAPREMONT_MODE_not specified',
 'FONDKAPREMONT_MODE_org spec account',
 'FONDKAPREMONT_MODE_reg oper account',
 'FONDKAPREMONT_MODE_reg oper spec account',
 'HOUSETYPE_MODE_block of flats',
 'HOUSETYPE_MODE_specific housing',
 'HOUSETYPE_MODE_terraced house',
 'WALLSMATERIAL_MODE_Block',
 'WALLSMATERIAL_MODE_Mixed',
 'WALLSMATERIAL_MODE_Monolithic',
 'WALLSMATERIAL_MODE_Others',
 'WALLSMATERIAL_MODE_Panel',
 'WALLSMATERIAL_MODE_Stone, brick',
 'WALLSMATERIAL_MODE_Wooden',
 'EMERGENCYSTATE_MODE_No',
 'EMERGENCYSTATE_MODE_Yes',
 'DAYS_EMPLOYED_PERC',
 'INCOME_CREDIT_PERC',
 'INCOME_PER_PERSON',
 'ANNUITY_INCOME_PERC',
 'PAYMENT_RATE',
 'BURO_DAYS_CREDIT_MIN',
 'BURO_DAYS_CREDIT_MAX',
 'BURO_DAYS_CREDIT_MEAN',
 'BURO_DAYS_CREDIT_VAR',
 'BURO_DAYS_CREDIT_ENDDATE_MIN',
 'BURO_DAYS_CREDIT_ENDDATE_MAX',
 'BURO_DAYS_CREDIT_ENDDATE_MEAN',
 'BURO_DAYS_CREDIT_UPDATE_MEAN',
 'BURO_CREDIT_DAY_OVERDUE_MAX',
 'BURO_CREDIT_DAY_OVERDUE_MEAN',
 'BURO_AMT_CREDIT_SUM_MAX',
 'BURO_AMT_CREDIT_SUM_MEAN',
 'BURO_AMT_CREDIT_SUM_SUM',
 'BURO_AMT_CREDIT_SUM_DEBT_MAX',
 'BURO_AMT_CREDIT_SUM_DEBT_MEAN',
 'BURO_AMT_CREDIT_SUM_DEBT_SUM',
 'BURO_AMT_CREDIT_SUM_OVERDUE_MEAN',
 'BURO_AMT_CREDIT_SUM_LIMIT_MEAN',
 'BURO_AMT_CREDIT_SUM_LIMIT_SUM',
 'BURO_CNT_CREDIT_PROLONG_SUM',
 'BURO_MONTHS_BALANCE_SIZE_SUM',
 'BURO_CREDIT_ACTIVE_Active_MEAN',
 'BURO_CREDIT_ACTIVE_Bad debt_MEAN',
 'BURO_CREDIT_ACTIVE_Closed_MEAN',
 'BURO_CREDIT_ACTIVE_Sold_MEAN',
 'BURO_CREDIT_ACTIVE_nan_MEAN',
 'BURO_CREDIT_CURRENCY_currency 1_MEAN',
 'BURO_CREDIT_CURRENCY_currency 2_MEAN',
 'BURO_CREDIT_CURRENCY_currency 3_MEAN',
 'BURO_CREDIT_CURRENCY_currency 4_MEAN',
 'BURO_CREDIT_CURRENCY_nan_MEAN',
 'BURO_CREDIT_TYPE_Another type of loan_MEAN',
 'BURO_CREDIT_TYPE_Car loan_MEAN',
 'BURO_CREDIT_TYPE_Cash loan (non-earmarked)_MEAN',
 'BURO_CREDIT_TYPE_Consumer credit_MEAN',
 'BURO_CREDIT_TYPE_Credit card_MEAN',
 'BURO_CREDIT_TYPE_Interbank credit_MEAN',
 'BURO_CREDIT_TYPE_Loan for business development_MEAN',
 'BURO_CREDIT_TYPE_Loan for purchase of shares (margin lending)_MEAN',
 'BURO_CREDIT_TYPE_Loan for the purchase of equipment_MEAN',
 'BURO_CREDIT_TYPE_Loan for working capital replenishment_MEAN',
 'BURO_CREDIT_TYPE_Microloan_MEAN',
 'BURO_CREDIT_TYPE_Mobile operator loan_MEAN',
 'BURO_CREDIT_TYPE_Mortgage_MEAN',
 'BURO_CREDIT_TYPE_Real estate loan_MEAN',
 'BURO_CREDIT_TYPE_Unknown type of loan_MEAN',
 'BURO_CREDIT_TYPE_nan_MEAN',
 'ACTIVE_DAYS_CREDIT_MIN',
 'ACTIVE_DAYS_CREDIT_MAX',
 'ACTIVE_DAYS_CREDIT_MEAN',
 'ACTIVE_DAYS_CREDIT_UPDATE_MEAN',
 'ACTIVE_CREDIT_DAY_OVERDUE_MAX',
 'ACTIVE_CREDIT_DAY_OVERDUE_MEAN',
 'ACTIVE_AMT_CREDIT_SUM_MAX',
 'ACTIVE_AMT_CREDIT_SUM_MEAN',
 'ACTIVE_AMT_CREDIT_SUM_SUM',
 'ACTIVE_AMT_CREDIT_SUM_DEBT_SUM',
 'ACTIVE_AMT_CREDIT_SUM_OVERDUE_MEAN',
 'ACTIVE_AMT_CREDIT_SUM_LIMIT_SUM',
 'ACTIVE_CNT_CREDIT_PROLONG_SUM',
 'ACTIVE_MONTHS_BALANCE_SIZE_SUM',
 'CLOSED_DAYS_CREDIT_MIN',
 'CLOSED_DAYS_CREDIT_MAX',
 'CLOSED_DAYS_CREDIT_MEAN',
 'CLOSED_DAYS_CREDIT_ENDDATE_MIN',
 'CLOSED_DAYS_CREDIT_ENDDATE_MAX',
 'CLOSED_DAYS_CREDIT_ENDDATE_MEAN',
 'CLOSED_DAYS_CREDIT_UPDATE_MEAN',
 'CLOSED_CREDIT_DAY_OVERDUE_MAX',
 'CLOSED_CREDIT_DAY_OVERDUE_MEAN',
 'CLOSED_AMT_CREDIT_SUM_MAX',
 'CLOSED_AMT_CREDIT_SUM_MEAN',
 'CLOSED_AMT_CREDIT_SUM_SUM',
 'CLOSED_AMT_CREDIT_SUM_DEBT_MAX',
 'CLOSED_AMT_CREDIT_SUM_DEBT_MEAN',
 'CLOSED_AMT_CREDIT_SUM_DEBT_SUM',
 'CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN',
 'CLOSED_AMT_CREDIT_SUM_LIMIT_SUM',
 'CLOSED_CNT_CREDIT_PROLONG_SUM',
 'CLOSED_MONTHS_BALANCE_SIZE_SUM',
 'PREV_AMT_ANNUITY_MIN',
 'PREV_AMT_ANNUITY_MAX',
 'PREV_AMT_ANNUITY_MEAN',
 'PREV_AMT_APPLICATION_MIN',
 'PREV_AMT_APPLICATION_MAX',
 'PREV_AMT_APPLICATION_MEAN',
 'PREV_AMT_CREDIT_MIN',
 'PREV_AMT_CREDIT_MAX',
 'PREV_AMT_CREDIT_MEAN',
 'PREV_APP_CREDIT_PERC_MIN',
 'PREV_APP_CREDIT_PERC_MAX',
 'PREV_APP_CREDIT_PERC_MEAN',
 'PREV_APP_CREDIT_PERC_VAR',
 'PREV_AMT_DOWN_PAYMENT_MIN',
 'PREV_AMT_DOWN_PAYMENT_MAX',
 'PREV_AMT_DOWN_PAYMENT_MEAN',
 'PREV_AMT_GOODS_PRICE_MIN',
 'PREV_AMT_GOODS_PRICE_MAX',
 'PREV_AMT_GOODS_PRICE_MEAN',
 'PREV_HOUR_APPR_PROCESS_START_MIN',
 'PREV_HOUR_APPR_PROCESS_START_MAX',
 'PREV_HOUR_APPR_PROCESS_START_MEAN',
 'PREV_RATE_DOWN_PAYMENT_MIN',
 'PREV_RATE_DOWN_PAYMENT_MAX',
 'PREV_RATE_DOWN_PAYMENT_MEAN',
 'PREV_DAYS_DECISION_MIN',
 'PREV_DAYS_DECISION_MAX',
 'PREV_DAYS_DECISION_MEAN',
 'PREV_CNT_PAYMENT_MEAN',
 'PREV_CNT_PAYMENT_SUM',
 'PREV_NAME_CONTRACT_TYPE_Cash loans_MEAN',
 'PREV_NAME_CONTRACT_TYPE_Consumer loans_MEAN',
 'PREV_NAME_CONTRACT_TYPE_Revolving loans_MEAN',
 'PREV_NAME_CONTRACT_TYPE_XNA_MEAN',
 'PREV_NAME_CONTRACT_TYPE_nan_MEAN',
 'PREV_WEEKDAY_APPR_PROCESS_START_FRIDAY_MEAN',
 'PREV_WEEKDAY_APPR_PROCESS_START_MONDAY_MEAN',
 'PREV_WEEKDAY_APPR_PROCESS_START_SATURDAY_MEAN',
 'PREV_WEEKDAY_APPR_PROCESS_START_SUNDAY_MEAN',
 'PREV_WEEKDAY_APPR_PROCESS_START_THURSDAY_MEAN',
 'PREV_WEEKDAY_APPR_PROCESS_START_TUESDAY_MEAN',
 'PREV_WEEKDAY_APPR_PROCESS_START_WEDNESDAY_MEAN',
 'PREV_WEEKDAY_APPR_PROCESS_START_nan_MEAN',
 'PREV_FLAG_LAST_APPL_PER_CONTRACT_N_MEAN',
 'PREV_FLAG_LAST_APPL_PER_CONTRACT_Y_MEAN',
 'PREV_FLAG_LAST_APPL_PER_CONTRACT_nan_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Building a house or an annex_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Business development_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Buying a garage_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Buying a home_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Buying a new car_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Buying a used car_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Car repairs_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Education_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Everyday expenses_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Furniture_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Gasification / water supply_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Hobby_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Journey_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Medicine_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Money for a third person_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Other_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Payments on other loans_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Purchase of electronic equipment_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Repairs_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Urgent needs_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_Wedding / gift / holiday_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_XAP_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_XNA_MEAN',
 'PREV_NAME_CASH_LOAN_PURPOSE_nan_MEAN',
 'PREV_NAME_CONTRACT_STATUS_Approved_MEAN',
 'PREV_NAME_CONTRACT_STATUS_Canceled_MEAN',
 'PREV_NAME_CONTRACT_STATUS_Refused_MEAN',
 'PREV_NAME_CONTRACT_STATUS_Unused offer_MEAN',
 'PREV_NAME_CONTRACT_STATUS_nan_MEAN',
 'PREV_NAME_PAYMENT_TYPE_Cash through the bank_MEAN',
 'PREV_NAME_PAYMENT_TYPE_Cashless from the account of the employer_MEAN',
 'PREV_NAME_PAYMENT_TYPE_Non-cash from your account_MEAN',
 'PREV_NAME_PAYMENT_TYPE_XNA_MEAN',
 'PREV_NAME_PAYMENT_TYPE_nan_MEAN',
 'PREV_CODE_REJECT_REASON_CLIENT_MEAN',
 'PREV_CODE_REJECT_REASON_HC_MEAN',
 'PREV_CODE_REJECT_REASON_LIMIT_MEAN',
 'PREV_CODE_REJECT_REASON_SCO_MEAN',
 'PREV_CODE_REJECT_REASON_SCOFR_MEAN',
 'PREV_CODE_REJECT_REASON_SYSTEM_MEAN',
 'PREV_CODE_REJECT_REASON_VERIF_MEAN',
 'PREV_CODE_REJECT_REASON_XAP_MEAN',
 'PREV_CODE_REJECT_REASON_XNA_MEAN',
 'PREV_CODE_REJECT_REASON_nan_MEAN',
 'PREV_NAME_TYPE_SUITE_Children_MEAN',
 'PREV_NAME_TYPE_SUITE_Family_MEAN',
 'PREV_NAME_TYPE_SUITE_Group of people_MEAN',
 'PREV_NAME_TYPE_SUITE_Other_A_MEAN',
 'PREV_NAME_TYPE_SUITE_Other_B_MEAN',
 'PREV_NAME_TYPE_SUITE_Spouse, partner_MEAN',
 'PREV_NAME_TYPE_SUITE_Unaccompanied_MEAN',
 'PREV_NAME_TYPE_SUITE_nan_MEAN',
 'PREV_NAME_CLIENT_TYPE_New_MEAN',
 'PREV_NAME_CLIENT_TYPE_Refreshed_MEAN',
 'PREV_NAME_CLIENT_TYPE_Repeater_MEAN',
 'PREV_NAME_CLIENT_TYPE_XNA_MEAN',
 'PREV_NAME_CLIENT_TYPE_nan_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Additional Service_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Animals_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Audio/Video_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Auto Accessories_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Clothing and Accessories_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Computers_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Construction Materials_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Consumer Electronics_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Direct Sales_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Education_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Fitness_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Furniture_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Gardening_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Homewares_MEAN',
 'PREV_NAME_GOODS_CATEGORY_House Construction_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Insurance_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Jewelry_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Medical Supplies_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Medicine_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Mobile_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Office Appliances_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Other_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Photo / Cinema Equipment_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Sport and Leisure_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Tourism_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Vehicles_MEAN',
 'PREV_NAME_GOODS_CATEGORY_Weapon_MEAN',
 'PREV_NAME_GOODS_CATEGORY_XNA_MEAN',
 'PREV_NAME_GOODS_CATEGORY_nan_MEAN',
 'PREV_NAME_PORTFOLIO_Cards_MEAN',
 'PREV_NAME_PORTFOLIO_Cars_MEAN',
 'PREV_NAME_PORTFOLIO_Cash_MEAN',
 'PREV_NAME_PORTFOLIO_POS_MEAN',
 'PREV_NAME_PORTFOLIO_XNA_MEAN',
 'PREV_NAME_PORTFOLIO_nan_MEAN',
 'PREV_NAME_PRODUCT_TYPE_XNA_MEAN',
 'PREV_NAME_PRODUCT_TYPE_walk-in_MEAN',
 'PREV_NAME_PRODUCT_TYPE_x-sell_MEAN',
 'PREV_NAME_PRODUCT_TYPE_nan_MEAN',
 'PREV_CHANNEL_TYPE_AP+ (Cash loan)_MEAN',
 'PREV_CHANNEL_TYPE_Car dealer_MEAN',
 'PREV_CHANNEL_TYPE_Channel of corporate sales_MEAN',
 'PREV_CHANNEL_TYPE_Contact center_MEAN',
 'PREV_CHANNEL_TYPE_Country-wide_MEAN',
 'PREV_CHANNEL_TYPE_Credit and cash offices_MEAN',
 'PREV_CHANNEL_TYPE_Regional / Local_MEAN',
 'PREV_CHANNEL_TYPE_Stone_MEAN',
 'PREV_CHANNEL_TYPE_nan_MEAN',
 'PREV_NAME_SELLER_INDUSTRY_Auto technology_MEAN',
 'PREV_NAME_SELLER_INDUSTRY_Clothing_MEAN',
 'PREV_NAME_SELLER_INDUSTRY_Connectivity_MEAN',
 'PREV_NAME_SELLER_INDUSTRY_Construction_MEAN',
 'PREV_NAME_SELLER_INDUSTRY_Consumer electronics_MEAN',
 'PREV_NAME_SELLER_INDUSTRY_Furniture_MEAN',
 'PREV_NAME_SELLER_INDUSTRY_Industry_MEAN',
 'PREV_NAME_SELLER_INDUSTRY_Jewelry_MEAN',
 'PREV_NAME_SELLER_INDUSTRY_MLM partners_MEAN',
 'PREV_NAME_SELLER_INDUSTRY_Tourism_MEAN',
 'PREV_NAME_SELLER_INDUSTRY_XNA_MEAN',
 'PREV_NAME_SELLER_INDUSTRY_nan_MEAN',
 'PREV_NAME_YIELD_GROUP_XNA_MEAN',
 'PREV_NAME_YIELD_GROUP_high_MEAN',
 'PREV_NAME_YIELD_GROUP_low_action_MEAN',
 'PREV_NAME_YIELD_GROUP_low_normal_MEAN',
 'PREV_NAME_YIELD_GROUP_middle_MEAN',
 'PREV_NAME_YIELD_GROUP_nan_MEAN',
 'PREV_PRODUCT_COMBINATION_Card Street_MEAN',
 'PREV_PRODUCT_COMBINATION_Card X-Sell_MEAN',
 'PREV_PRODUCT_COMBINATION_Cash_MEAN',
 'PREV_PRODUCT_COMBINATION_Cash Street: high_MEAN',
 'PREV_PRODUCT_COMBINATION_Cash Street: low_MEAN',
 'PREV_PRODUCT_COMBINATION_Cash Street: middle_MEAN',
 'PREV_PRODUCT_COMBINATION_Cash X-Sell: high_MEAN',
 'PREV_PRODUCT_COMBINATION_Cash X-Sell: low_MEAN',
 'PREV_PRODUCT_COMBINATION_Cash X-Sell: middle_MEAN',
 'PREV_PRODUCT_COMBINATION_POS household with interest_MEAN',
 'PREV_PRODUCT_COMBINATION_POS household without interest_MEAN',
 'PREV_PRODUCT_COMBINATION_POS industry with interest_MEAN',
 'PREV_PRODUCT_COMBINATION_POS industry without interest_MEAN',
 'PREV_PRODUCT_COMBINATION_POS mobile with interest_MEAN',
 'PREV_PRODUCT_COMBINATION_POS mobile without interest_MEAN',
 'PREV_PRODUCT_COMBINATION_POS other with interest_MEAN',
 'PREV_PRODUCT_COMBINATION_POS others without interest_MEAN',
 'PREV_PRODUCT_COMBINATION_nan_MEAN',
 'APPROVED_AMT_ANNUITY_MIN',
 'APPROVED_AMT_ANNUITY_MAX',
 'APPROVED_AMT_ANNUITY_MEAN',
 'APPROVED_AMT_APPLICATION_MIN',
 'APPROVED_AMT_APPLICATION_MAX',
 'APPROVED_AMT_APPLICATION_MEAN',
 'APPROVED_AMT_CREDIT_MIN',
 'APPROVED_AMT_CREDIT_MAX',
 'APPROVED_AMT_CREDIT_MEAN',
 'APPROVED_APP_CREDIT_PERC_MIN',
 'APPROVED_APP_CREDIT_PERC_MAX',
 'APPROVED_APP_CREDIT_PERC_MEAN',
 'APPROVED_AMT_DOWN_PAYMENT_MIN',
 'APPROVED_AMT_DOWN_PAYMENT_MAX',
 'APPROVED_AMT_DOWN_PAYMENT_MEAN',
 'APPROVED_AMT_GOODS_PRICE_MIN',
 'APPROVED_AMT_GOODS_PRICE_MAX',
 'APPROVED_AMT_GOODS_PRICE_MEAN',
 'APPROVED_HOUR_APPR_PROCESS_START_MIN',
 'APPROVED_HOUR_APPR_PROCESS_START_MAX',
 'APPROVED_HOUR_APPR_PROCESS_START_MEAN',
 'APPROVED_RATE_DOWN_PAYMENT_MIN',
 'APPROVED_RATE_DOWN_PAYMENT_MAX',
 'APPROVED_RATE_DOWN_PAYMENT_MEAN',
 'APPROVED_DAYS_DECISION_MIN',
 'APPROVED_DAYS_DECISION_MAX',
 'APPROVED_DAYS_DECISION_MEAN',
 'APPROVED_CNT_PAYMENT_MEAN',
 'APPROVED_CNT_PAYMENT_SUM',
 'POS_MONTHS_BALANCE_MAX',
 'POS_MONTHS_BALANCE_MEAN',
 'POS_MONTHS_BALANCE_SIZE',
 'POS_SK_DPD_MAX',
 'POS_SK_DPD_MEAN',
 'POS_SK_DPD_DEF_MAX',
 'POS_SK_DPD_DEF_MEAN',
 'POS_NAME_CONTRACT_STATUS_Active_MEAN',
 'POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN',
 'POS_NAME_CONTRACT_STATUS_Approved_MEAN',
 'POS_NAME_CONTRACT_STATUS_Canceled_MEAN',
 'POS_NAME_CONTRACT_STATUS_Completed_MEAN',
 'POS_NAME_CONTRACT_STATUS_Demand_MEAN',
 'POS_NAME_CONTRACT_STATUS_Returned to the store_MEAN',
 'POS_NAME_CONTRACT_STATUS_Signed_MEAN',
 'POS_NAME_CONTRACT_STATUS_XNA_MEAN',
 'POS_NAME_CONTRACT_STATUS_nan_MEAN',
 'POS_COUNT',
 'INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE',
 'INSTAL_DPD_MAX',
 'INSTAL_DPD_MEAN',
 'INSTAL_DPD_SUM',
 'INSTAL_DBD_MAX',
 'INSTAL_DBD_MEAN',
 'INSTAL_DBD_SUM',
 'INSTAL_PAYMENT_PERC_MAX',
 'INSTAL_PAYMENT_PERC_MEAN',
 'INSTAL_PAYMENT_PERC_SUM',
 'INSTAL_PAYMENT_PERC_VAR',
 'INSTAL_PAYMENT_DIFF_MAX',
 'INSTAL_PAYMENT_DIFF_MEAN',
 'INSTAL_PAYMENT_DIFF_SUM',
 'INSTAL_PAYMENT_DIFF_VAR',
 'INSTAL_AMT_INSTALMENT_MAX',
 'INSTAL_AMT_INSTALMENT_MEAN',
 'INSTAL_AMT_INSTALMENT_SUM',
 'INSTAL_AMT_PAYMENT_MIN',
 'INSTAL_AMT_PAYMENT_MAX',
 'INSTAL_AMT_PAYMENT_MEAN',
 'INSTAL_AMT_PAYMENT_SUM',
 'INSTAL_DAYS_ENTRY_PAYMENT_MAX',
 'INSTAL_DAYS_ENTRY_PAYMENT_MEAN',
 'INSTAL_DAYS_ENTRY_PAYMENT_SUM',
 'INSTAL_COUNT']

In [56]:
data_preprocessed = data.copy()
data_preprocessed = data_preprocessed[cols_to_keep]
print(len(cols_to_keep), data_preprocessed.shape[1])

541 541


In [57]:
data_preprocessed['NAME_INCOME_TYPE_Maternity leave'] = 0
data_preprocessed['NAME_FAMILY_STATUS_Unknown leave'] = 0

In [58]:
data_preprocessed.shape[1]

543

## Outliers

In [59]:
temp = data_preprocessed.describe()
temp.loc['max'].max()

inf

In [61]:
# Drop inf values
data_preprocessed.replace([np.inf, -np.inf], np.nan, inplace=True)

In [63]:
# Check
temp = data_preprocessed.describe()
temp.loc['max'].max()

157985144608.63922

## Missing data

In [64]:
missing_values_summary(data_preprocessed, ascending=False)

,index,Null_counts,Non_Null_counts,Percentage_Null
253,ACTIVE_AMT_CREDIT_SUM_MEAN,14081,34663,28.887658
252,ACTIVE_AMT_CREDIT_SUM_MAX,14081,34663,28.887658
247,ACTIVE_DAYS_CREDIT_MAX,14079,34665,28.883555
258,ACTIVE_CNT_CREDIT_PROLONG_SUM,14079,34665,28.883555
257,ACTIVE_AMT_CREDIT_SUM_LIMIT_SUM,14079,34665,28.883555
...,...,...,...,...
134,ORGANIZATION_TYPE_Industry: type 1,0,48744,0.000000
133,ORGANIZATION_TYPE_Housing,0,48744,0.000000
132,ORGANIZATION_TYPE_Hotel,0,48744,0.000000
131,ORGANIZATION_TYPE_Government,0,48744,0.000000


In [66]:
print(data_preprocessed.shape)
data_preprocessed.dropna(inplace=True)
print(data_preprocessed.shape)

(48744, 543)
(17361, 543)


# Sample

In [67]:
sample = data_preprocessed[:1000]
sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 3 to 2763
Columns: 543 entries, SK_ID_CURR to NAME_FAMILY_STATUS_Unknown leave
dtypes: float64(368), int64(44), uint8(131)
memory usage: 3.3 MB


In [69]:
file_path = os.path.join(parent_dir, "data", "sample.csv")
sample.to_csv(file_path, index=False)